## This notebook will investigate right of way for some specific networks

In [5]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')
import network as nw
import road as rd
import traffic_lights as tl
import torch
import FV_schemes as fv
import junction as jn
import initial_and_bc as ibc

### 2-1 junction

We consider the simplest junction in which there is the need for a priority parameter.
In this case it is only necessary with one priority parameter.

The specific network has two incoming roads and one outgoing road. There is only one outgoing road, and hence no crossing connections. Assume that road one has priority over road 2.

The distribution matrix takes the form
$$
A = 
\begin{bmatrix}
1 \\
1
\end{bmatrix}.
$$

For all cases we will assume the roads have lengths of 50 meters.

In [2]:
# Configuration of the network
distribution = [[1.0], [1.0]]
priorities = [[1], [2]]
crossing_connections =  [[[]],
                        [[]]]
L = 50
N = 5

#### Case 1:
Two equivalent roads with speed limits 50 km/h leading into a road with speed limit 50km/h. Almost no traffic in the beginning, but the combined influx into the two roads is bigger than the capacity of the outgoing road, so that there will arise some congestion.

For the first case we have no traffic lights.

In [7]:
# Creating the network
boundary_1 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.3]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary_2 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.3]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.1, id = "1_fw", boundary_fnc=boundary_1)
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, -1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.1, id = "2_fw", boundary_fnc=boundary_2)
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.2, id = "3_fw")

entering = [0,1]
leaving = [2]

junction = jn.Junction([road1, road2, road3], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 50
network = nw.RoadNetwork([road1, road2, road3], [junction], T)


In [8]:
densities, _, _, _ = network.solve_cons_law()

In [5]:
# Comment out to create the gif - note this will crash the kernel
# Need to update gif creation code to create this gif...

#### Case 1:
Two equivalent roads with speed limits 50 km/h leading into a road with speed limit 50km/h. Here there is much traffic in the beginning, but the influx to the roads is less than the capacity of the outgoing road. Expected behaviour: traffic on road 1 should clear out before the traffic on the second road.

We still have no traffic lights.

In [9]:
# Creating the network
boundary_fnc_1 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.1]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary_fnc_2 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.1]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, id = "1_fw", boundary_fnc=boundary_fnc_1)
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, -1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, id = "2_fw", boundary_fnc=boundary_fnc_2)
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.2, id = "3_fw")

entering = [0,1]
leaving = [2]

junction = jn.Junction([road1, road2, road3], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 100
network = nw.RoadNetwork([road1, road2, road3], [junction], T)


In [10]:
densities, _, _, _ = network.solve_cons_law()

In [11]:
# Comment out to create the gif - note this will crash the kernel
# Update gif drawing script to correctly draw this gif

### 2-2 junction

This is the smallest junction in which there may be crossing connections. Order the roads 1, 2, 3, 4, with 1 and 2 being the incoming roads, and 3 and 4 being the outgoing roads. Assume that road 1 has right of way. That means that it has priority going in to the two outgoing roads. In addition, road 2 crosses connection 1->3 when going to outgoing road 4. Hence, an upper bound needs to be calculated. Since there are two outgoing roads, we need a distribution matrix A. For the first case, assume that A takes the form
$$
A = 
\begin{bmatrix}
1 & 0\\
0 & 1
\end{bmatrix}
$$
that is, all traffic on road 1 continues to road 3 and all traffic on road 2 goes to road 4.


In [12]:
# Configuration of the network
distribution = [[1.0, 0.0], [0.0, 1.0]]
priorities = [[1, 1], [2, 2]]
crossing_connections =  [[[], []],
                        [[], [(0,0)]]]
L = 50
N = 5

Case 1:

All roads have the same maximum densities, and the same speed limits of 50 km/h. We assume that there are no traffic lights. Assume there is quite a lot of traffic on both roads, and not that high inflow of traffic.

In [13]:
# Creating the network
boundary1 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.3]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary2 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.05]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 0), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, id = "1_fw", boundary_fnc=boundary1)
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 1), right_pos = (0, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "2_fw", boundary_fnc=boundary2)
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "3_fw")
road4 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0), right_pos = (0, -1),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "4_fw")

entering = [0,1]
leaving = [2, 3]

junction = jn.Junction([road1, road2, road3, road4], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 150
network = nw.RoadNetwork([road1, road2, road3, road4], [junction], T)

In [15]:
densities, queues, _, _ = network.solve_cons_law()

In [ ]:
# glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = '2-2-row.gif')

### 4-4 junction
This is the largest junction in the simulation, and we would therefore like to investigate some of the phenoma arising in this junction. Each road in the simulation is unidirectional, meaning that to be able to model a bi-directional road, we model it as two individual uni-directional roads. The junction we are considering is really only a 2-2 junction where each road is bi-directional, but for us we will model them as a 4-4 junction. We order the roads as following:
- road 1 goes from left to right into the junction
- road 2 goes from left to right out from the junction
- road 3 goes from right to left into the junction
- road 4 goes from rigth to left out from the junction
- road 5 goes from top to bottom into the junction
- road 6 goes from top to bottom out from the junction
- road 7 goes from bottom to top into the junction
- road 8 goes from bottom to top out from the junction

We will assume that the road going from left to right (and opposite) has right of way. We will also assume that most of the traffic will keep going, and that some of it will turn left or right. In addition to roads 1,2,3 and 4 having right of way, we will also assume a right hand rule where applicable. We summarize the distribution of traffic in the distribution matrix A:
$$
A = 
\begin{bmatrix}
0.8 & 0.0 & 0.1 & 0.1\\
0.0 & 0.8 & 0.1 & 0.1\\
0.25 & 0.25 & 0.5 & 0.0\\
0.25 & 0.25 & 0.0 & 0.5
\end{bmatrix}.
$$ 
We assume that u-turns are not allowed, which is why some of the terms are equal to zero.

Now for the crossing connections. Traffic from road 1 crosses no lanes when going to roads 2 and 6. However, when going to road 8, traffic from road 3 to road 4 is being crossed.
Traffic from road 3 crosses no lanes when going to road 4 and road 8. However, when going to road 6, traffic from road 1 to road 2 is being crossed.
Traffic from road 5 crosses no lanes when going to road 4. Traffic going to road 2 crosses traffic from road 3 to road 4, and traffic going to road 6 crosses traffic from road 3 to road 4, traffic from 1 to 2 and traffic from road 1 to road 8.
Traffic from road 7 crosses no lanes when going to road 2. Traffic  going to lane 4 crosses traffic from road 1 to road 2, and traffic going to road 8 crosses traffic from road 1 to road 2, road 3 to road 4 and road 3 to road 6. 
We summarize the crossing connections in the matrix $C$:
$$
C = 
\begin{bmatrix}
. & . & . & [(1,1)]\\
. & . & [(0,0)] & .\\
[(1,1)] & . & [(0,0),(1,1),(0,3)] & .\\
. & [(0,0)] & . & [(0,0), (1,1), (1,2)]
\end{bmatrix}
$$
We also need to specify the prirorities of the incoming roads for each outgoing road. This we summarize in the matrix $P$:
$$
P = 
\begin{bmatrix}
1 & 0 & 1 & 2\\
0 & 1 & 2 & 1\\
3 & 2 & 3 & 0\\
2 & 3 & 0 & 3
\end{bmatrix},
$$
here elements equal to zero indicate that the edge is not allowed. The positions of these elements should match with the zero elements of the distribution matrix.
We still assume that there are no traffic lights in the junction.

---

% (0,0) : 1 -> 2 (0,1) : 1 -> 4 (1,0) : 3 -> 2 (2,2) : 5 -> 6

### Test case 1

In [16]:
# Configuration of the network
# distribution = [[0.9, 0.0, 0.05, 0.05],
#                 [0.0, 0.9, 0.05, 0.05],
#                 [0.25, 0.25, 0.5, 0.0],
#                 [0.25, 0.25, 0.0, 0.5]]
distribution = [[0.9, 0.0, 0.1, 0.0],
                [0.0, 0.9, 0.0, 0.1],
                [0.05, 0.05, 0.9, 0.0],
                [0.05, 0.05, 0.0, 0.9]]
priorities = [[1, 0, 1, 2],
              [0, 1, 2, 1],
              [3, 2, 3, 0],
              [2, 3, 0, 3]]
crossing_connections =  [[[], [], [], [(1,1)]],
                         [[], [], [(0,0)], []],
                         [[(1,1)], [], [(0,0),(1,1),(0,3)], []],
                         [[], [(0,0)], [], [(0,0),(1,1),(1,2)]]]

L = 50
N = 5

In [17]:
torch.autograd.set_detect_anomaly(False)

In [18]:
# Creating the network
boundary1 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.3]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary2 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.3]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary3 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.12]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary4 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.12]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 0), right_pos = (-0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, id = "1_fw", boundary_fnc=boundary1)
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0.1, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "2_fw")
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (1, 0), right_pos = (0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, id = "3_bw", boundary_fnc=boundary2)
road4 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-0.1, 0), right_pos = (-1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3,  id = "4_bw")
road5 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 1), right_pos = (0, 0.1),
                initial = lambda x: torch.ones_like(x) * 0.4, id = "5_fw", boundary_fnc=boundary3)
road6 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -0.1), right_pos = (0, -1),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "6_fw")
road7 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -1), right_pos = (0, -0.1),
                initial = lambda x: torch.ones_like(x) * 0.4, id = "7_bw", boundary_fnc=boundary4)
road8 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0.1), right_pos = (0, 1),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "8_bw")

entering = [0, 2, 4, 6]
leaving = [1, 3, 5, 7]

junction = jn.Junction([road1, road2, road3, road4, road5, road6, road7, road8], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 200
network = nw.RoadNetwork([road1, road2, road3, road4, road5, road6, road7, road8], [junction], T)

In [19]:
densities, queues, _, _ = network.solve_cons_law()

In [20]:
times = densities[0].keys()
objective = torch.tensor(0)
for i in range(len(densities)):
    for t in times:
        objective = objective + torch.sum(densities[i][t])

In [21]:
objective.backward()

Checking if autograd still works

In [22]:
# print(queues)
# glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = '4-4-test.gif')
# Check that equilibrium is correct...

#### Test case 2

In [23]:
distribution = [[0.7, 0.0, 0.15, 0.15],
                [0.0, 0.7, 0.15, 0.15],
                [0.3, 0.3, 0.4, 0.0],
                [0.3, 0.3, 0.0, 0.4]]
priorities = [[1, 0, 1, 2],
              [0, 1, 2, 1],
              [3, 2, 3, 0],
              [2, 3, 0, 3]]
crossing_connections =  [[[], [], [], [(1,1)]],
                         [[], [], [(0,0)], []],
                         [[(1,1)], [], [(0,0),(1,1),(0,3)], []],
                         [[], [(0,0)], [], [(0,0),(1,1),(1,2)]]]

L = 50
N = 5

In [24]:
# Creating the network
boundary1 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.3]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary2 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.3]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary3 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.12]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary4 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.12]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 0), right_pos = (-0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, id = "1_fw", boundary_fnc=boundary1)
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0.1, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "2_fw")
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (1, 0), right_pos = (0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, id = "3_bw", boundary_fnc=boundary2)
road4 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-0.1, 0), right_pos = (-1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3,  id = "4_bw")
road5 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 1), right_pos = (0, 0.1),
                initial = lambda x: torch.ones_like(x) * 0.4, id = "5_fw", boundary_fnc=boundary3)
road6 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -0.1), right_pos = (0, -1),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "6_fw")
road7 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -1), right_pos = (0, -0.1),
                initial = lambda x: torch.ones_like(x) * 0.4, id = "7_bw", boundary_fnc=boundary4)
road8 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0.1), right_pos = (0, 1),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "8_bw")

entering = [0, 2, 4, 6]
leaving = [1, 3, 5, 7]

junction = jn.Junction([road1, road2, road3, road4, road5, road6, road7, road8], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 50
network = nw.RoadNetwork([road1, road2, road3, road4, road5, road6, road7, road8], [junction], T)

In [25]:
densities, queues, _, _ = network.solve_cons_law()

In [ ]:
# glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = '4-4-test2.gif')

#### Test case 3

In [26]:
distribution = [[1.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, 0.0],
                [0.0, 0.0, 1.0, 0.0],
                [0.0, 0.0, 0.0, 1.0]]
priorities = [[1, 0, 1, 2],
              [0, 1, 2, 1],
              [3, 2, 3, 0],
              [2, 3, 0, 3]]
crossing_connections =  [[[], [], [], [(1,1)]],
                         [[], [], [(0,0)], []],
                         [[(1,1)], [], [(0,0),(1,1),(0,3)], []],
                         [[], [(0,0)], [], [(0,0),(1,1),(1,2)]]]

L = 50
N = 5

In [27]:
# Creating the network
boundary1 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.15]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary2 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.15]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary3 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.1]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
boundary4 = ibc.boundary_conditions(1, max_dens = 1, densities = torch.tensor([0.1]),
                                           time_jumps = [], in_speed = torch.tensor(50.0),
                                           L = L)
road1 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-1, 0), right_pos = (-0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, id = "1_fw", boundary_fnc=boundary1)
road2 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0.1, 0), right_pos = (1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "2_fw")
road3 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (1, 0), right_pos = (0.1, 0),
                initial = lambda x: torch.ones_like(x) * 0.7, id = "3_bw", boundary_fnc=boundary2)
road4 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (-0.1, 0), right_pos = (-1, 0),
                initial = lambda x: torch.ones_like(x) * 0.3,  id = "4_bw")
road5 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 1), right_pos = (0, 0.1),
                initial = lambda x: torch.ones_like(x) * 0.4, id = "5_fw", boundary_fnc=boundary3)
road6 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -0.1), right_pos = (0, -1),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "6_fw")
road7 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, -1), right_pos = (0, -0.1),
                initial = lambda x: torch.ones_like(x) * 0.4, id = "7_bw", boundary_fnc=boundary4)
road8 = rd.Road(1, L, N, torch.tensor([50.0], requires_grad=True), [], left_pos = (0, 0.1), right_pos = (0, 1),
                initial = lambda x: torch.ones_like(x) * 0.3, id = "8_bw")

entering = [0, 2, 4, 6]
leaving = [1, 3, 5, 7]

junction = jn.Junction([road1, road2, road3, road4, road5, road6, road7, road8], entering, leaving, distribution, trafficlights=[],
                       coupled_trafficlights = [], duty_to_gw = True, priorities = priorities,
                       crossing_connections = crossing_connections)
T = 150
network = nw.RoadNetwork([road1, road2, road3, road4, road5, road6, road7, road8], [junction], T)

In [28]:
densities, queues, _, _ = network.solve_cons_law()

In [ ]:
# glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = '4-4-test3.gif')

End of simulation reached!
Saving GIF as: 4-4-test3.gif


: 